In [1]:
from transformers import BertTokenizer, BertForSequenceClassification, Trainer, TrainingArguments
from transformers import AutoModelForSequenceClassification, AutoTokenizer, Trainer, TrainingArguments

from sklearn.model_selection import train_test_split
import pandas as pd
import torch
import json


In [29]:
f = open('translated_chatgpt.json')
  
# returns JSON object as 
# a dictionary
data = json.load(f)
df_ai = pd.DataFrame(data)

In [30]:
import re

with open("trnews-64/trnews-64.train.raw") as fi:
    articles = re.split("\n\n", fi.read()) 

In [31]:
articles = articles[0:1000]

In [32]:
truncated = []
for article in articles:
    splitted = article.split(" ")
    truncated.append(splitted[0:200])


In [33]:
truncated_text_human = []
for trunc in truncated:
    truncated_text_human.append(" ".join(trunc))

In [34]:
df_human = pd.DataFrame(truncated_text_human, columns = ['Text'])

In [35]:
df_human = df_human.drop(44)

In [36]:
df_human['label'] = 0

In [37]:
df_ai['label'] = 1

In [38]:
data = pd.concat([df_ai, df_human])


In [39]:
len(data)

2001

In [40]:
df_shuffled = data.sample(frac=1).reset_index(drop=True)

df = df_shuffled

In [41]:
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from snowballstemmer import TurkishStemmer
  

nltk.download('stopwords')
nltk.download('punkt')

# Define Turkish stopwords
stop_words = set(stopwords.words('turkish'))

# Define Turkish stemmer
#stemmer = SnowballStemmer('turkish')
stemmer=TurkishStemmer()

def preprocess_text(text):
    # Tokenize the text into individual words
    tokens = word_tokenize(text.lower())

    # Remove stop words
    tokens = [token for token in tokens if token not in stop_words]

    # Stem the remaining words
    tokens = [stemmer.stemWord(token) for token in tokens]

    # Rejoin the words into a single string
    preprocessed_text = ' '.join(tokens)

    return preprocessed_text


[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/bahadir/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /Users/bahadir/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [42]:
from nltk.tokenize import word_tokenize
max_words = 200
df['Text'] = df['Text'].apply(preprocess_text)
df['Text'] = df['Text'].apply(lambda x: ' '.join(word_tokenize(x)[:max_words]))



In [43]:
from sklearn.model_selection import train_test_split

train, test = train_test_split(df, test_size=0.2, stratify = df['label'])

In [44]:
train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1600 entries, 363 to 1712
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   Text    1600 non-null   object
 1   label   1600 non-null   int64 
dtypes: int64(1), object(1)
memory usage: 37.5+ KB


In [45]:
data = []
for index, row in train.iterrows():
    label = '__label__' + str(row['label'])
    text = row['Text']
    example = label + ' ' + text
    data.append(example)

# Save the examples to a text file
with open('fasttext_train_data.txt', 'w') as f:
    for example in data:
        f.write(example + '\n')
        
data = []
for index, row in test.iterrows():
    label = '__label__' + str(row['label'])
    text = row['Text']
    example = label + ' ' + text
    data.append(example)
print(len(data))
# Save the examples to a text file
with open('fasttext_test_data.txt', 'w') as f:
    for example in data:
        f.write(example + '\n')

401


In [46]:
import fasttext
# load the training data
train_data = "fasttext_train_data.txt"
# load the testing data
test_data = "fasttext_test_data.txt"

# train the model
model = fasttext.train_supervised(input=train_data, epoch=25,lr=1.0)

# evaluate the model on the testing data
result = model.test(test_data)

# print the evaluation metrics
print("Number of examples:", result[0])
print("Precision:", result[1])
print("Recall:", result[2])



Read 0M words
Number of words:  24234
Number of labels: 2
Progress:  49.2% words/sec/thread: 4126576 lr:  0.508127 avg.loss:  0.091350 ETA:   0h 0m 0s

Number of examples: 401
Precision: 0.9551122194513716
Recall: 0.9551122194513716


Progress: 100.0% words/sec/thread: 2988046 lr:  0.000000 avg.loss:  0.049777 ETA:   0h 0m 0s


In [47]:
model = fasttext.train_supervised(input=train_data, epoch=50)

# evaluate the model on the testing data
result = model.test(test_data)

# print the evaluation metrics
print("Number of examples:", result[0])
print("Precision:", result[1])
print("Recall:", result[2])


Read 0M words
Number of words:  24234
Number of labels: 2
Progress:  92.3% words/sec/thread: 4238240 lr:  0.007675 avg.loss:  0.106826 ETA:   0h 0m 0s

Number of examples: 401
Precision: 0.9675810473815462
Recall: 0.9675810473815462


Progress: 100.0% words/sec/thread: 3680048 lr:  0.000000 avg.loss:  0.099812 ETA:   0h 0m 0s


In [48]:
model = fasttext.train_supervised(input=train_data, epoch=5000)

# evaluate the model on the testing data
result = model.test(test_data)

# print the evaluation metrics
print("Number of examples:", result[0])
print("Precision:", result[1])
print("Recall:", result[2])

Read 0M words
Number of words:  24234
Number of labels: 2
Progress: 100.0% words/sec/thread: 4321983 lr:  0.000047 avg.loss:  0.002381 ETA:   0h 0m 0s0.004318 ETA:   0h 0m28s

Number of examples: 401
Precision: 0.9675810473815462
Recall: 0.9675810473815462


Progress: 100.0% words/sec/thread: 4313967 lr:  0.000000 avg.loss:  0.002381 ETA:   0h 0m 0s
